In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd 
import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

import keras
from keras.models import Sequential
from keras.layers import Dense

import matplotlib.pyplot as plt

In [ ]:
filepath = "/content/drive/MyDrive/Dataset-Unicauca-Version2-87Atts.csv"
df = pd.read_csv(filepath)

In [ ]:
df=df.iloc[:100000]

In [ ]:
feats_importance = ['Destination.IP', 'Destination.Port', 'Source.IP', 'Init_Win_bytes_forward',
 'min_seg_size_forward', 'Fwd.Packet.Length.Max', 'Init_Win_bytes_backward', 'Flow.IAT.Max',
 'Source.Port', 'Flow.Duration', 'Fwd.Packet.Length.Std', 'Bwd.IAT.Total',
 'Avg.Fwd.Segment.Size', 'Fwd.Packets.s', 'Fwd.IAT.Total', 'Fwd.IAT.Max',
 'Fwd.Packet.Length.Mean', 'Subflow.Fwd.Bytes', 'Flow.Bytes.s', 'Min.Packet.Length',
 'Total.Length.of.Fwd.Packets', 'Bwd.IAT.Max', 'Packet.Length.Variance', 'Bwd.Packets.s',
 'Flow.IAT.Mean', 'Fwd.Header.Length', 'act_data_pkt_fwd', 'Max.Packet.Length',
 'Flow.Packets.s', 'Flow.IAT.Std', 'Packet.Length.Std', 'Idle.Max',
 'Fwd.Header.Length.1', 'Bwd.Packet.Length.Mean', 'Bwd.IAT.Std', 'Fwd.Packet.Length.Min',
 'Bwd.Packet.Length.Std', 'Avg.Bwd.Segment.Size', 'Average.Packet.Size', 'Total.Length.of.Bwd.Packets',
 'Packet.Length.Mean', 'Fwd.IAT.Mean', 'Fwd.IAT.Std', 'Flow.IAT.Min',
 'Bwd.IAT.Mean', 'Bwd.Packet.Length.Max', 'Subflow.Fwd.Packets', 'Total.Fwd.Packets',
 'Total.Backward.Packets', 'Bwd.Header.Length', 'Subflow.Bwd.Bytes', 'Subflow.Bwd.Packets', 
 'Idle.Mean', 'Fwd.IAT.Min', 'Down.Up.Ratio', 'Idle.Min']

In [ ]:
from sklearn.preprocessing import LabelEncoder
number = LabelEncoder()
df['Label'] = number.fit_transform(df['Label'].astype(str))
df['Flow.ID'] = number.fit_transform(df['Flow.ID'].astype(str))
df['Source.IP'] = number.fit_transform(df['Source.IP'].astype(str))
df['Destination.IP'] = number.fit_transform(df['Destination.IP'].astype(str))
df['Timestamp'] = number.fit_transform(df['Timestamp'].astype(str))
df['ProtocolName'] = number.fit_transform(df['ProtocolName'].astype(str))	

In [ ]:
feats = [x for x in df.columns if x != 'ProtocolName']
X = df[feats].astype(float)
Y = df['ProtocolName']

In [ ]:
dummy_y = np_utils.to_categorical(Y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, dummy_y, test_size = 0.3, random_state = 42)

In [ ]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
import keras.backend as K
def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [ ]:
input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv1D, MaxPool1D, Flatten, Input
inp =  Input(shape=(86,1))
conv = Conv1D(filters=2, kernel_size=2)(inp)
pool = MaxPool1D(pool_size=2)(conv)
flat = Flatten()(pool)
dense = Dense(1)(flat)
model = Model(inp, dense)
model.compile(loss='mae', optimizer='adam',metrics=['acc'])

print(model.summary())
model.fit(X_train,y_train,epochs = 5,batch_size = 30)

In [ ]:
model.compile(loss='mae', optimizer='adam', metrics=['acc'])

In [ ]:
%%time
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

In [ ]:
%%time
loss, accuracy = model.evaluate(X_test, y_test, verbose=2)

In [ ]:
print("Accuracy:")
print(accuracy)


In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['Training', 'Testing'], loc='upper left')
plt.savefig("Accuracy_img.png")

In [ ]:
plt.plot(history.history['loss']) 
plt.plot(history.history['val_loss']) 
plt.title('Model loss') 
plt.ylabel('Loss') 
plt.xlabel('Epochs') 
plt.legend(['Training', 'Testing'], loc='upper left') 
plt.savefig("Loss_img.png")

In [ ]:

X_train=np.expand_dims(X_train, axis=2)
y_train=np.expand_dims(y_train,axis=1)


In [ ]:

from tensorflow.keras.layers import SimpleRNN
model = Sequential()
model.add(SimpleRNN(64, activation='relu', input_shape=(86,1)))
model.add(Dense(1, activation='relu'))


print('Training...')
model.compile(loss='mse', optimizer='adam',metrics=['accuracy'])
print (model.summary())
print ('\n')



history = model.fit(X_train,y_train, epochs=20, batch_size=1, verbose=1)
RNN=model.evaluate(X_train,y_train,verbose=1)[1]*100
print("RNN Accuracy",RNN,'%')


In [ ]:
import numpy as np
import pandas as pd
import gc
import os

from sklearn.preprocessing import RobustScaler, normalize
from sklearn.model_selection import train_test_split, KFold, GroupKFold
from sklearn.metrics import mean_absolute_error


import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.layers import Bidirectional, LSTM, Multiply
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.layers import Concatenate, Add, GRU
from tensorflow.keras.layers import BatchNormalization



from scipy.signal import butter, filtfilt
from pickle import dump

from IPython.display import display
from warnings import filterwarnings
filterwarnings('ignore')


In [ ]:
def dnn_model():
    
    x_input = Input(shape=(X_train.shape[-2:]))
    
    x1 = Bidirectional(LSTM(units=768, return_sequences=True))(x_input)
    x2 = Bidirectional(LSTM(units=512, return_sequences=True))(x1)
    x3 = Bidirectional(LSTM(units=384, return_sequences=True))(x2)
    x4 = Bidirectional(LSTM(units=256, return_sequences=True))(x3)
    x5 = Bidirectional(LSTM(units=128, return_sequences=True))(x4)
    
    z2 = Bidirectional(GRU(units=384, return_sequences=True))(x2)
    
    z31 = Multiply()([x3, z2])
    z31 = BatchNormalization()(z31)
    z3 = Bidirectional(GRU(units=256, return_sequences=True))(z31)
    
    z41 = Multiply()([x4, z3])
    z41 = BatchNormalization()(z41)
    z4 = Bidirectional(GRU(units=128, return_sequences=True))(z41)
    
    z51 = Multiply()([x5, z4])
    z51 = BatchNormalization()(z51)
    z5 = Bidirectional(GRU(units=64, return_sequences=True))(z51)
    
    x = Concatenate(axis=2)([x5, z2, z3, z4, z5])
    
    x = Dense(units=128, activation='selu')(x)
    
    x_output = Dense(units=1)(x)

    model = Model(inputs=x_input, outputs=x_output, 
                  name='DNN_Model')
    return model

In [ ]:
model = dnn_model()
model.summary()

In [ ]:
history = model.fit(X_train,y_train, epochs=20, batch_size=1, verbose=1)
DNN=model.evaluate(X_train,y_train,verbose=1)[1]*100
print("DNN Accuracy",RNN,'%')